In [116]:
import pandas as pd
import numpy as np
import seaborn as sns

In [117]:
smcl_data = pd.read_csv("SMCL_ANF_Items.csv")

C:\Users\rho62\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [118]:

#Renamed the columns for column V, W, X

smcl_data2 = smcl_data.rename(columns={'245|a': 'Book Title','100|a': 'Author',
                                      '008 Date One': 'Published Date'})

#dropped columns

smcl_data_clean = smcl_data2.drop(['LOCATION', 'BARCODE', 'RECORD #(ITEM)',
                 'I TYPE', 'MAT TYPE', 'OTHER CALL', 'MATERIAL'], axis=1)

#created dictionary for the BRANCH column and replaced with actual location

branch_dict = {'0k': 'Bookmobile', '1a':'Atherton', '1b':'Belmont', '1r':'Brisbane','1e':'East Palo Alto','1f':'Foster City'
,'1h':'Half Moon Bay','1m':'Millbrae','1p':'Pacifica Sharp Park','1z':'Pacifica Sanchez','1s':'San Carlos','1v':'Portola Valley','1w':'Woodside'}

#looking for values to combine in Call # field
smcl_data_clean["CALL #"].value_counts()

#cleaning up and only using initial categorey 
smcl_data_clean["CALL #"] = smcl_data_clean["CALL #"].str.split(" ", 1, expand=True)

smcl_data_clean = smcl_data_clean.replace({"BRANCH": branch_dict})

In [119]:
#looking for values to combine in Call # field
smcl_data_clean["CALL #"].value_counts()

#cleaning up and only using initial categorey 
smcl_data_clean["CALL #"] = smcl_data_clean["CALL #"].str.split(" ", 1, expand=True)
smcl_data_clean.head()

,BRANCH,CREATED(ITEM),RECORD #(BIBLIO),CREATED(BIBLIO),YTDCIRC,LYRCIRC,TOT CHKOUT,TOT RENEW,# RENEWALS,STATUS,DUE DATE,LOUTDATE,LCHKIN,CALL #,Book Title,Author,Published Date
0,Portola Valley,7/18/2005 8:45,b10000203,7/18/2005 8:45,0,0,15,6,0,m,- -,4/28/2016 15:35,5/24/2016 11:00,NATURE,Encyclopedia of mammals /,NaN,1998
1,Pacifica Sanchez,7/18/2005 8:45,b10000276,7/18/2005 8:45,0,0,30,11,0,-,- -,4/18/2018 17:17,5/14/2018 10:40,NATURE,Encyclopedia of fishes /,NaN,1998
2,Foster City,7/18/2005 8:45,b1000032x,7/18/2005 8:45,2,3,40,23,0,-,- -,11/24/2019 16:41,1/10/2020 10:47,NATURE,Encyclopedia of birds /,NaN,1998
3,Millbrae,7/18/2005 8:45,b1000032x,7/18/2005 8:45,0,0,17,9,0,-,- -,7/12/2018 15:36,7/14/2018 11:13,NATURE,Encyclopedia of birds /,NaN,1998
4,East Palo Alto,7/18/2005 8:47,b10002364,7/18/2005 8:47,0,0,8,8,0,m,- -,6/8/2016 16:57,8/17/2016 17:01,COOKING,Caribbean and African cooking /,"Grant, Rosamund.",1998


In [120]:
#Looking to see which roll to remove
smcl_data_clean["CALL #"].value_counts()

BIO        6789
COOKING    5231
HISTORY    4966
HEALTH     4575
TRAVEL     4540
           ... 
F-PER         1
RO-KLE        1
F-ROB         1
F-HEN         1
F-HAI         1
Name: CALL #, Length: 254, dtype: int64

In [121]:
#Removing items starting with "F-"
smcl_data_removed1 = smcl_data_clean[~smcl_data_clean["CALL #"].astype(str).str.startswith('F-')]
#Removing items starting with "M-"
smcl_data_removed2 = smcl_data_removed1[~smcl_data_removed1["CALL #"].astype(str).str.startswith('M-')]
#Removing items starting with "SF-"
smcl_data_removed3 = smcl_data_removed2[~smcl_data_removed2["CALL #"].astype(str).str.startswith('SF-')]
#Removing items starting with "YA-"
smcl_data_removed4 = smcl_data_removed3[~smcl_data_removed3["CALL #"].astype(str).str.startswith('YA-')]

#Removing items that appear less than 10 times
smcl_data_removed5 = smcl_data_removed4.groupby('CALL #').filter(lambda d: len(d) > 10)

#Removing items that are incorrectly placed into ANF
smcl_data_removed6 = smcl_data_removed5[smcl_data_removed5["CALL #"] != "USB"]
smcl_data_removed7 = smcl_data_removed6[smcl_data_removed6["CALL #"] != "SP"]
smcl_data_removed8 = smcl_data_removed7[smcl_data_removed7["CALL #"] != "CHI"]
smcl_data_removed9 = smcl_data_removed8[smcl_data_removed8["CALL #"] != "J"]
smcl_data_removed10 = smcl_data_removed9[smcl_data_removed9["CALL #"] != "DVD"]
smcl_data_removed11 = smcl_data_removed10[smcl_data_removed10["CALL #"] != "CD"]


In [140]:
#Combining Call # that are the same
smcl_data_removed_final = smcl_data_removed11.replace({
                                                        "SOC": "SOC SCI",
                                                        "PERS": "PERS FIN", "PER": "PERS FIN",
                                                        "SPORTS": "SPORT",
                                                        "LANG": "LANG INST",
                                                        "CRAFTS": "CRAFT",
                                                        "PETS": "PET",
                                                        "SELF": "SELFHELP",
                                                        "LITERATUR": "LIT",
                                                        "PHILOSOPH": "PHILSOPH",
                                                        })

In [137]:
TOT_CIRC = smcl_data_removed_final["TOT CHKOUT"] + smcl_data_removed_final["TOT RENEW"]
smcl_data_removed_final["TOT CIRC"] =  TOT_CIRC

smcl_data_df = smcl_data_removed_final[["BRANCH", "Book Title", "Author", "Published Date", "CALL #", "TOT CHKOUT", "TOT RENEW", "TOT CIRC", "STATUS", "DUE DATE", "LOUTDATE", "LCHKIN", "CREATED(ITEM)"]]
smcl_data_df.head()

,BRANCH,Book Title,Author,Published Date,CALL #,TOT CHKOUT,TOT RENEW,TOT CIRC,STATUS,DUE DATE,LOUTDATE,LCHKIN,CREATED(ITEM)
0,Portola Valley,Encyclopedia of mammals /,NaN,1998,NATURE,15,6,21,m,- -,4/28/2016 15:35,5/24/2016 11:00,7/18/2005 8:45
1,Pacifica Sanchez,Encyclopedia of fishes /,NaN,1998,NATURE,30,11,41,-,- -,4/18/2018 17:17,5/14/2018 10:40,7/18/2005 8:45
2,Foster City,Encyclopedia of birds /,NaN,1998,NATURE,40,23,63,-,- -,11/24/2019 16:41,1/10/2020 10:47,7/18/2005 8:45
3,Millbrae,Encyclopedia of birds /,NaN,1998,NATURE,17,9,26,-,- -,7/12/2018 15:36,7/14/2018 11:13,7/18/2005 8:45
4,East Palo Alto,Caribbean and African cooking /,"Grant, Rosamund.",1998,COOKING,8,8,16,m,- -,6/8/2016 16:57,8/17/2016 17:01,7/18/2005 8:47


In [158]:
smcl_crosstable = smcl_data_df.pivot_table(values="TOT CIRC", index=["CALL #"], columns="BRANCH", aggfunc='mean')
smcl_crosstable.round(2)

BRANCH,Atherton,Belmont,Bookmobile,Brisbane,East Palo Alto,Foster City,Half Moon Bay,Millbrae,Pacifica Sanchez,Pacifica Sharp Park,Portola Valley,San Carlos,Woodside
CALL #,,,,,,,,,,,,,
ART,16.12,35.56,19.00,20.86,14.14,38.61,25.62,30.71,30.09,26.57,28.63,22.26,21.18
BIO,19.33,17.40,8.29,12.32,9.87,21.01,15.42,18.07,19.22,20.06,18.00,16.33,18.63
BUSINESS,24.43,35.37,14.33,16.51,14.34,35.54,20.95,27.42,22.12,19.83,23.55,26.49,15.46
COLLECT,4.60,7.60,NaN,8.33,2.40,10.26,7.54,9.62,7.04,6.14,4.09,13.96,7.67
COMPUTE,33.00,38.33,NaN,20.62,5.10,47.39,20.96,28.12,29.41,27.82,24.30,28.37,15.58
CONSUMER,NaN,19.00,NaN,17.40,0.00,18.88,0.00,0.00,10.75,20.00,40.00,40.50,17.00
COOKING,35.05,43.18,32.00,25.65,23.74,48.65,40.82,42.13,40.05,35.82,35.19,35.28,31.53
CRAFT,29.25,37.98,1.00,30.53,23.78,44.54,31.98,35.50,32.41,36.23,28.83,30.76,27.07
EDUCATE,19.44,27.46,0.00,12.50,10.09,31.03,12.14,23.00,14.31,14.11,17.92,21.73,12.30
